In [1]:
import socket
import time
import json
import threading
import pandas as pd
import re
import sys
import numpy as np

#### This is a visualization of our DNS Zones

![zones_telematik.png](zones_telematik.png)

### This class is representing a DNS server and is handing all requests and responses to a DNS server

In [2]:
class NpEncoder(json.JSONEncoder):
    def default(self, obj):
        if isinstance(obj, np.integer):
            return int(obj)
        elif isinstance(obj, np.floating):
            return float(obj)
        elif isinstance(obj, np.ndarray):
            return obj.tolist()
        else:
            return super(NpEncoder, self).default(obj)
        
class DNS_server(object):
    
    #socket = None
#     address = None
#     port = None
    #is_authorative = 0 #is this server an authorative server?
    #is_root = 0        #is this server a root server?
    #dns_zone = None    #own dns zone as json
    #records_in_own_zone = None #all ns and a records in own zone. (this information is alreay 
                                    #conrained in the dns_zone variable but we are keeping this sepearate variable for now )  
    def __init__(self, zone_number, authorative, is_root, seconds=2000,addr='127.0.0.11', port=53053): 
        self.address = addr
        self.port    = port
        self.socket = socket.socket(socket.AF_INET, socket.SOCK_DGRAM)
        self.is_root = is_root
        self.is_authorative = authorative
        self.dns_zone, self.records_in_own_zone = self.load_zone(zone_number)

    '''loads all entrys for own zone from our json file. 
    (e.g. zonename: telematik for zone (telematik, swith, router))'''
    def load_zone(self, zone_number):
        ## load all entries for my zone
        with open('names.json') as dnsnamedata:
            data = json.load(dnsnamedata)
            dns_zone = data[zone_number]
        
        telematik_dns_dict = {'ns': pd.DataFrame(dns_zone['ns'])['host'].tolist(),
                              'a':pd.DataFrame(dns_zone['a'])['value'].tolist(),
                              'ttl':pd.DataFrame(dns_zone['a'])['ttl'].tolist()}
        records_in_own_zone = pd.DataFrame(telematik_dns_dict, columns = ['ns','a','ttl' ])
        return dns_zone, records_in_own_zone
       # print('loads the zones')
    
    '''checks if a requested entry is in own zone. 
    (e.g. this zonename is telematik and requested address is for dns name www.swith.telematik.
    Means that swith is in our zone and we can send the adress for switch.telematik)
    Returns None if not. '''
    def is_in_own_zone(self, addr):
        #names_i_know = pd.DataFrame(data[0]['ns'])['host'].tolist()
        
        print('check if x is in our zone')
        
        ns = None
        a = None
        ttl = None
        for counter,name in enumerate (self.records_in_own_zone['ns']):
            #define pattern somestring.name
            pattern = re.compile(r'(.*?)(\.'+str(name)+')?$')

            #apply the addr we are looking for to the patter and return a list with results
            # e.g. ['www.switch', '.telematik'] for mattching or ['www.switch.telematik1', None] for not matching
            groups = pattern.match(addr).groups()
            if(groups[1] != None): 
                ns = self.records_in_own_zone['ns'][counter]
                a = self.records_in_own_zone['a'][counter]
                ttl = self.records_in_own_zone['ttl'][counter]
        return ns, a, ttl
        #return telematik_dns_df[telematik_dns_df['ns'] == forward_to]
    
    #check if this dns can send directly the authorative answer?
    #def have_authorative_answer():
    
    def build_response(self, msg_json):
        payload= {}
        msg = json.loads(msg_json)
        if not msg['dns.flags.response']:  # if a question set this flag to 1 (answer)
            payload['dns.flags.response'] = 1
        else:
            payload['dns.flags.response'] = 0  # do nothing??????????????????????
        payload['dns.flags.recavail'] = 0
        payload['dns.qry.name'] = msg['dns.qry.name']
        payload['dns.qry.type'] =  msg['dns.qry.type']                # 1steht für a record "????????????????????????????????????????"
        ns, a, ttl  = self.is_in_own_zone(msg['dns.qry.name'])
        if a == None:
            payload['dns.flags.rcode'] = 3
        else:
            payload['dns.flags.rcode'] = 0
        payload['dns.count.answers'] = 1 # es können auch mehrere ip's einen dns vertreten!!!
        payload['dns.flags.authoritative'] = self.is_authorative
        payload['dns.a'] = a
        payload['dns.rsp.ttl.a'] = ttl
        payload['dns.ns'] = ns
        return payload
    
#     def build_dataframe():
        
        
#     def write_log():
    def response_to_client(payload, client_addr):
        
        try:
            payload['dns.flags.response'] = 'Changed!'
            payload_json = json.dumps(payload)
            time.sleep(0.5)
            server_socket.sendto(bytes(payload_json, 'utf-8'), client_addr)
            # nachricht kommt rein 
            # nachricht wird geschickt
        except:
            print('could not respond to client: ',client_addr)
        
    def send_request(self, ip, port, payload):
        self.socket.sendto(bytes(payload, 'utf-8'),(ip, port))
        print("message successfull sended to ip:{} port:{}\n".format(ip,port), payload)
    #this method is to start a thread to listen and handle requests and responses.
    def start(self):
        
        server_addr = (self.address, self.port)
        self.socket.bind(server_addr)
        while True:
            payload_from_dns, client_addr = self.socket.recvfrom(1024)
            #print(json.loads(payload_from_dns))
            payload = self.build_response(payload_from_dns)
            #print(payload)
            client_ip = client_addr[0]
            client_port = client_addr[1]
            #print(type(payload))
#             try:
           # tt =json.dumps(payload, cls=NpEncoder)
#             except:
#                 e = sys.exc_info()[0]
            #print(payload)
            #print(tt)
            self.send_request(client_ip, client_port, json.dumps(payload, cls=NpEncoder, indent=4))
            
    
        

In [3]:
t = DNS_server(0,authorative=0,is_root=0)
resolver_thread = threading.Thread(target=t.start)
resolver_thread.start()

In [4]:
a = {'dns.flags.response': 1, 'dns.flags.recavail': 0, 'dns.qry.name': '.switch.telematik', 'dns.qry.type': 1, 'dns.flags.rcode': 0, 'dns.count.answers': 1, 'dns.flags.authoritative': 0, 'dns.a': '255.255.255.255', 'dns.rsp.ttl.a': 400, 'dns.ns': 'switch.telematik'}

check if x is in our zone
message successfull sended to ip:127.0.0.10 port:53053
 {
    "dns.flags.response": 1,
    "dns.flags.recavail": 0,
    "dns.qry.name": ".switch.telematik",
    "dns.qry.type": 1,
    "dns.flags.rcode": 0,
    "dns.count.answers": 1,
    "dns.flags.authoritative": 0,
    "dns.a": "255.255.255.255",
    "dns.rsp.ttl.a": 400,
    "dns.ns": "switch.telematik"
}
check if x is in our zone
message successfull sended to ip:127.0.0.10 port:53053
 {
    "dns.flags.response": 1,
    "dns.flags.recavail": 0,
    "dns.qry.name": ".switch.telematik",
    "dns.qry.type": 1,
    "dns.flags.rcode": 0,
    "dns.count.answers": 1,
    "dns.flags.authoritative": 0,
    "dns.a": "255.255.255.255",
    "dns.rsp.ttl.a": 400,
    "dns.ns": "switch.telematik"
}


In [7]:
json.dumps(a)

'{"dns.flags.response": 1, "dns.flags.recavail": 0, "dns.qry.name": ".switch.telematik", "dns.qry.type": 1, "dns.flags.rcode": 0, "dns.count.answers": 1, "dns.flags.authoritative": 0, "dns.a": "255.255.255.255", "dns.rsp.ttl.a": 400, "dns.ns": "switch.telematik"}'

In [9]:
msg ={
    "dns.flags.response":0,
    "dns.flags.recdesired":1,
    "dns.qry.name":".switch.telematik",
    "dns.qry.type":1
}

In [12]:
msg_json = json.dumps(msg, indent=4)

In [13]:
t.build_response(msg_json)

check if x is in our zone


{'dns.flags.response': 1,
 'dns.flags.recavail': 0,
 'dns.qry.name': '.switch.telematik',
 'dns.qry.type': 1,
 'dns.flags.rcode': 0,
 'dns.count.answers': 1,
 'dns.flags.authoritative': 0,
 'dns.a': '255.255.255.255',
 'dns.rsp.ttl.a': 400,
 'dns.ns': 'switch.telematik'}

In [84]:
payload_dict ={  
    "dns.flags.response": 1,
    "dns.flags.recdesired": "placeholder",
    "dns.qry.name": "placeholder",
    "dns.qry.type": "placeholder",
    "dns.flags.rcode": "placeholder",
    "dns.count.answers": "placeholder",
    "dns.flags.authoritative": "placeholder",
    "dns.a": "127.0.0.10",
    "dns.ns": "www.switch.telematik",
    "dns.resp.ttl": 1234,
    "dns.srv.name": "placeholder",
    "dns.srv.port": "placeholder",
    "dns.srv.proto": "placeholder",
    "dns.srv.service": "placeholder",
    "dns.srv.target": "placeholder"
}  

In [82]:
payload_dict

{'dns.flags.response': 1,
 'dns.flags.recdesired': 'placeholder',
 'dns.qry.name': 'placeholder',
 'dns.qry.type': 'placeholder',
 'dns.flags.rcode': 'placeholder',
 'dns.count.answers': 'placeholder',
 'dns.flags.authoritative': 'placeholder',
 'dns.a': '127.0.0.10',
 'dns.ns': 'www.switch.telematik',
 'dns.resp.ttl': 1234,
 'dns.srv.name': 'placeholder',
 'dns.srv.port': 'placeholder',
 'dns.srv.proto': 'placeholder',
 'dns.srv.service': 'placeholder',
 'dns.srv.target': 'placeholder'}

In [ ]:
response_dict ={  
    "dns.flags.response": "1", #Boolean, Frage oder Antwort
    "dns.flags.recavail": "0",#"dns.flags.recdesired": "placeholder", #Boolean Rekursion oder nicht -> antwort dns.flags.recavail =0 antwort vom authorativen server
    "dns.qry.name": "wwlnawrocki.tns",    #Character string name der zu anfragen den seite wwlnawrocki.tns
    "dns.qry.type": "placeholder", # 	Unsigned integer, 2 bytes, 1= a record, 0 anscheinend nicht
    
    "dns.flags.rcode": "placeholder", # Unsigned integer, 2 bytes response code is 0=alles ist k
    "dns.count.answers": "placeholder", # Unsigned integer, 2 bytes wie viele antwortne=1
    "dns.flags.authoritative": "placeholder", # Boolean  die antwort ist autoritative
    
    "dns.a": " 127.0,0.101", #IPv4 address      dns.a =
    
    "dns.ns": "", #Character string   ttl : 300 , falls dns.qry.type=2
    
    "dns.resp.ttl": 1234, #ttl : 300
    
    
    
    
    
    
    "dns.srv.name": "placeholder",  #Character string
    "dns.srv.port": "placeholder",  #Unsigned integer, 2 bytes
    "dns.srv.proto": "placeholder",  #Character string
    "dns.srv.service": "placeholder", #Character string
    "dns.srv.target": "placeholder"  #Character string
}  

In [69]:
t =DNS_server(0,authorative=0,is_root=0)

In [70]:
t.load_zone()
t.dns_zone

TypeError: load_zone() missing 1 required positional argument: 'zone_number'

In [71]:
# t = [x for x in t.records_in_own_zone["ns"]]

In [72]:
t.is_in_own_zone('.router.telematik')

check if x is in our zone


('.router.telematik', '127.0.0.13')

In [73]:
pattern = re.compile(r'(.*?)(\.'+str('switch.telematik')+')?$')

In [30]:
pattern

re.compile(r'(.*?)(\.switch.telematik)?$', re.UNICODE)

In [27]:
pattern = re.compile(r'(.*?)(\.'+str("test")+')?$')

In [36]:
pattern.match('mail.switch.telematik').groups()

('mail', '.switch.telematik')

In [26]:
z2.t

5

In [27]:
z2.t = 3

In [28]:
z2.t

3

In [29]:
z.t

5

In [30]:
z.t

5

In [37]:
test.t

6

In [32]:
z2.t

3

In [143]:

city = pd.DataFrame([['Sacramento', 'California'], ['Miami', 'Florida']], columns=['City', 'State'])


In [147]:
city

,City,State
0,Sacramento,California
1,Miami,Florida


In [148]:
city.to_csv('city2.csv')

In [172]:
payload_dict ={  
    "dns.flags.response": 0,
    "dns.flags.recdesired": "placeholder",
    "dns.qry.name": "placeholder",
    "dns.qry.type": "placeholder",
    "dns.flags.rcode": "placeholder",
    "dns.count.answers": "placeholder",
    "dns.flags.authoritative": "placeholder",
    "dns.a": "127.0.0.10",
    "dns.ns": "www.switch.telematik",
    "dns.resp.ttl": 1234,
    "dns.srv.name": "placeholder",
    "dns.srv.port": "placeholder",
    "dns.srv.proto": "placeholder",
    "dns.srv.service": "placeholder",
    "dns.srv.target": "placeholder"
}  

In [173]:
keys = list(payload_dict.keys())

In [174]:
values = list(payload_dict.values())

In [175]:
testos = pd.DataFrame(values, keys)

In [168]:
testos.to_csv('city23.csv')

In [176]:
col_names = keys
titanic_data = pd.read_csv('city23.csv', names=col_names, header=None)

In [171]:
titanic_data

,dns.flags.response,dns.flags.recdesired,dns.qry.name,dns.qry.type,dns.flags.rcode,dns.count.answers,dns.flags.authoritative,dns.a,dns.ns,dns.resp.ttl,dns.srv.name,dns.srv.port,dns.srv.proto,dns.srv.service,dns.srv.target
0,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,dns.flags.response,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,dns.flags.recdesired,placeholder,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,dns.qry.name,placeholder,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,dns.qry.type,placeholder,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,dns.flags.rcode,placeholder,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,dns.count.answers,placeholder,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,dns.flags.authoritative,placeholder,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,dns.a,127.0.0.10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,dns.ns,www.switch.telematik,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
